<a href="https://colab.research.google.com/github/aashnadogra/github_actions/blob/main/CharGrad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-generativeai diffusers transformers accelerate safetensors pandas torch tqdm pillow torch clip-anytorch cmaes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [ ]:
import json
import csv
import re
import time
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import google.generativeai as genai
import torch
from clip import clip
from cmaes import CMA
import matplotlib.font_manager as fm
from diffusers import StableDiffusionPipeline

In [ ]:


# Configuration
genai.configure(api_key='')
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)
font_paths = fm.findSystemFonts(fontpaths=None, fontext='ttf')

arial_font_path = next((path for path in font_paths if 'arial' in path.lower()), None)
if not arial_font_path:
    arial_font_path = next((path for path in font_paths if 'liberation' in path.lower() and 'sans' in path.lower()), None)

if arial_font_path:
    FONT_PATH = arial_font_path
else:
    raise FileNotFoundError("Arial font not found on the system. Please install it.")
FONT_SIZE = 24
CHARACTERS = ['o', 'i', 'l', 's', 't', 'e', 'a']
REPLACEMENTS = {
    'o': ['0', 'O'], 'i': ['1', 'I'], 'l': ['1', '|'],
    's': ['5', 'S'], 't': ['7', 'T'], 'e': ['3', 'E'],
    'a': ['4', 'A']
}

pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float32)
pipe.to(device)


In [ ]:
# Character rendering and similarity calculation
def render_character(char):
    """Render character to pixel array"""
    image = Image.new('L', (FONT_SIZE, FONT_SIZE), 255)
    draw = ImageDraw.Draw(image)
    font = ImageFont.truetype(FONT_PATH, FONT_SIZE)
    draw.text((0, 0), char, font=font, fill=0)
    return np.array(image)

def jaccard_similarity(a, b):
    """Calculate pixel similarity using Jaccard index"""
    intersection = np.logical_and(a < 128, b < 128)
    union = np.logical_or(a < 128, b < 128)
    return intersection.sum() / union.sum()

# Precompute character similarities
CHAR_SIMILARITY = {}
for orig in CHARACTERS:
    char_map = {}
    orig_pixels = render_character(orig)
    for repl in REPLACEMENTS[orig]:
        repl_pixels = render_character(repl)
        char_map[repl] = jaccard_similarity(orig_pixels, repl_pixels)
    CHAR_SIMILARITY[orig] = char_map


In [ ]:
# Core implementation components
class CharGradAttack:
    def __init__(self, target_term, max_iter=5, population_size=10):
        self.target_term = target_term
        self.max_iter = max_iter
        self.pop_size = population_size
        self.optimizer = CMA(mean=np.zeros(len(target_term)), sigma=0.5, population_size=population_size)

    def compute_gradient(self, prompt):
        """Evolutionary strategy-based gradient estimation"""
        population = []
        for _ in range(self.pop_size):
            candidate = self._perturb_candidate(prompt)
            # Evaluate the candidate and add to population
            eval_score = self._evaluate_candidate(candidate)
            population.append((candidate, eval_score))

        # Sort population by evaluation score (lower is better)
        population.sort(key=lambda x: x[1])

        # Ensure we provide exactly pop_size solutions to tell()
        solutions = []
        for i in range(self.pop_size):
            # If we have fewer than pop_size solutions, duplicate the best ones
            idx = min(i, len(population) - 1)
            cand, score = population[idx]
            solutions.append((self._vectorize(cand), score))

        # Update optimizer with solutions
        self.optimizer.tell(solutions)

        # Get new candidates
        return self.optimizer.ask()

    def _perturb_candidate(self, prompt):
        """Generate perturbed candidate using similarity-guided replacements"""
        words = prompt.split()
        noun = words[0]
        perturbed_noun = []

        for i, c in enumerate(noun):
            if c.lower() in CHAR_SIMILARITY:
                replacements = CHAR_SIMILARITY[c.lower()]
                best_repl = max(replacements, key=lambda x: replacements[x])
                perturbed_noun.append(best_repl.upper() if c.isupper() else best_repl)
            else:
                perturbed_noun.append(c)

        return ' '.join([''.join(perturbed_noun)] + words[1:])

    def _evaluate_candidate(self, candidate):
      """CLIP-based loss calculation"""
      # Generate images using Stable Diffusion
      original_img = pipe(self.target_term).images[0]
      candidate_img = pipe(candidate).images[0]

      # Move images to the same device as the clip_model
      # Convert PIL Image to PyTorch tensor
      original_img = preprocess(original_img).unsqueeze(0).to(device)
      candidate_img = preprocess(candidate_img).unsqueeze(0).to(device)

      original_embed = clip_model.encode_image(original_img)
      candidate_embed = clip_model.encode_image(candidate_img)

      return 1 - torch.cosine_similarity(original_embed, candidate_embed).item()

    def _vectorize(self, text):
        """Convert text to optimization vector"""
        # Extract just the noun part (first word)
        noun = text.split()[0]
        # Pad or truncate to match target_term length
        if len(noun) < len(self.target_term):
            noun = noun.ljust(len(self.target_term))
        elif len(noun) > len(self.target_term):
            noun = noun[:len(self.target_term)]
        return np.array([ord(c) for c in noun])


In [8]:
# Pipeline implementation
model = genai.GenerativeModel("gemini-1.5-flash")

def generate_prompts(num_prompts=100):
    """Generate base prompts following <noun> in <background> format"""
    response = model.generate_content(f"""
        Generate {num_prompts} unique prompts in JSON array format.
        Each prompt must follow exactly: "<noun> in <background>"
        Example: ["elephant in space", "flowers in desert"]
    """)

    match = re.search(r'\[.*?\]', response.text, re.DOTALL)
    return json.loads(match.group()) if match else []

def attack_pipeline():
    # Generate base prompts
    base_prompts = generate_prompts(100)
    results = []

    for prompt in base_prompts:
        target_noun = prompt.split()[0]
        attacker = CharGradAttack(target_noun)

        best_adv = prompt
        min_similarity = float('inf')

        for _ in range(attacker.max_iter):
            candidate_vectors = attacker.compute_gradient(prompt)

            # Process each candidate vector
            for i in range(len(candidate_vectors)):
                vector = candidate_vectors[i]
                words = prompt.split()

                # Safely convert vector elements to characters
                safe_chars = []
                for c in vector:
                    # Ensure the value is in valid ASCII range
                    char_value = max(32, min(126, int(c)))
                    safe_chars.append(chr(char_value))

                # Create perturbed prompt (replace just the noun)
                candidate_text = ' '.join([''.join(safe_chars)] + words[1:])

                try:
                    similarity = attacker._evaluate_candidate(candidate_text)

                    if similarity < min_similarity:
                        best_adv = candidate_text
                        min_similarity = similarity
                except Exception as e:
                    print(f"Error evaluating candidate: {e}")
                    continue

        results.append({
            'normal': prompt,
            'adversarial': best_adv,
            'similarity': min_similarity
        })
        time.sleep(1)  # Rate limit handling

    # Save results
    with open('prompts.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=['normal', 'adversarial', 'similarity'])
        writer.writeheader()
        writer.writerows(results)

if __name__ == "__main__":
    attack_pipeline()

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

TypeError: 'numpy.float64' object is not iterable

In [12]:
import torch
from diffusers import StableDiffusionXLPipeline
import pandas as pd
import os
from tqdm import tqdm

# Set high precision for matrix operations
torch.set_float32_matmul_precision('high')

# Create output directories
os.makedirs("normal_images", exist_ok=True)
os.makedirs("adversarial_images", exist_ok=True)

# Load the Stable Diffusion XL model
model = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    cache_dir="./models/",
    torch_dtype=torch.float16,
    use_auth_token=True,
    safety_checker=None,
).to("cuda")

# Optimize model inference
model.unet = torch.compile(model.unet, mode="reduce-overhead", fullgraph=True)

# Read prompts from CSV
df = pd.read_csv("/content/prompts (1).csv")  # Ensure correct CSV path
normal_prompts = df.iloc[:, 0].dropna().tolist()  # Column 0: Normal prompts
adversarial_prompts = df.iloc[:, 1].dropna().tolist()  # Column 1: Adversarial prompts

# Ensure both lists have the same length
min_len = min(len(normal_prompts), len(adversarial_prompts))
normal_prompts = normal_prompts[:min_len]
adversarial_prompts = adversarial_prompts[:min_len]

# Generate images for both normal and adversarial prompts
for i, (normal_prompt, adv_prompt) in tqdm(
    enumerate(zip(normal_prompts, adversarial_prompts), start=1),
    total=len(normal_prompts),
    desc="Generating images"
):
    try:
        # Generate Normal Image
        print(f"Generating Normal Image {i}: {normal_prompt}")
        normal_image = model(prompt=normal_prompt, num_inference_steps=50).images[0]
        normal_image.save(f"normal_images/prompt_{i}.png")

        # Generate Adversarial Image
        print(f"Generating Adversarial Image {i}: {adv_prompt}")
        adversarial_image = model(prompt=adv_prompt, num_inference_steps=50).images[0]
        adversarial_image.save(f"adversarial_images/prompt_{i}.png")

    except Exception as e:
        print(f"Error generating images for prompt {i}: {e}")

print("Image generation complete!")


Keyword arguments {'use_auth_token': True, 'safety_checker': None} are not expected by StableDiffusionXLPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Generating images:   0%|          | 0/87 [00:00<?, ?it/s]

Generating Normal Image 1: Astronauts in a neon-lit cyberpunk city


  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

Generating Adversarial Image 87: CenT4urS in a Grecian landscape


  0%|          | 0/50 [00:00<?, ?it/s]

Generating images: 100%|██████████| 87/87 [17:18<00:00, 11.93s/it]

Image generation complete!
